In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import numpy as np
import pandas as pd
from glob import glob
from tqdm import tqdm
import matplotlib.pyplot as plt
import pathlib



In [3]:
class HyperParametersSummary:
    def __init__(self, filter_path: str, test_fname='final_test_miou.txt'):
        self.predictions = []
        self.parameters = []
        self.table = []
        self.test_fname = test_fname
        for f in tqdm(glob(filter_path)):
            parameters = f.split('/')[-1]
            self.parameters.append(parameters)
            column = {}
            for parameter in parameters.split(','):                
                parameter_key, parameter_value = parameter.split('=')
                column[parameter_key] = parameter_value
            # Insert some code here to parse the metrics and their values
            test_file = os.path.join(f, 'testing', "fo=%s" %column["fold"], self.test_fname)
            if os.path.isfile(test_file):
                with open(test_file) as f:
                    content = f.readlines()
                    if len(content) > 0:
                        content = content[0].split(',')
                        for line in content[:1]:
                            key, value = line.split(':')
                            column[key] = float(value)
            else:
                pass
            self.table.append(column)
        self.table = pd.DataFrame(self.table)
        self.group_names = ', '.join(self.table.columns)
        print(f'Parameters: {self.group_names}')


## MS COCO

In [4]:
exp_dir = "experiments/200113_152702_n_shots_fold_embed_type_bs_seed_milestone_length_num_epoch_split_model_type_dataset_name_coco_1n5shots/*"

hp_summary = HyperParametersSummary(exp_dir, test_fname='final_test_miou_1.txt')

results_aggregated = hp_summary.table.groupby(['dataset_name', 'split', 'model_type', 'fold']).mean().unstack()
results_aggregated['mIOU'] = results_aggregated.mean(axis=1)

results_aggregated

100%|██████████| 160/160 [00:00<00:00, 1077.92it/s]

Parameters: milestone_length, num_epoch, n_shots, model_type, fold, seed, split, bs, embed_type, dataset_name, IOU


IOU                               \
fold                                         0        1         2         3   
dataset_name split    model_type                                              
coco         trainval coatt           0.121020  0.13496  0.118800  0.048540   
                      iter_nwe_coatt  0.132560  0.13234  0.041375  0.077575   
                      nwe             0.128320  0.12490  0.068300  0.067425   
                      nwe_coatt       0.153975  0.14844  0.060040  0.065220   

                                          mIOU  
fold                                            
dataset_name split    model_type                
coco         trainval coatt           0.105830  
                      iter_nwe_coatt  0.095963  
                      nwe             0.097236  
                      nwe_coatt       0.106919

In [5]:
exp_dir = "experiments/200107_143436_n_shots_fold_embed_type_bs_seed_milestone_length_num_epoch_split_model_type_dataset_name_coco_iternwe_epochs_milestone/*"

hp_summary = HyperParametersSummary(exp_dir, test_fname='final_test_miou_1.txt')

results_aggregated = hp_summary.table.groupby(['dataset_name', 'split', 'model_type', 'num_epoch', 'milestone_length', 'fold']).mean().unstack()
results_aggregated['mIOU'] = results_aggregated.mean(axis=1)

results_aggregated

100%|██████████| 180/180 [00:00<00:00, 354.96it/s]

Parameters: milestone_length, num_epoch, n_shots, model_type, fold, seed, split, bs, embed_type, dataset_name, IOU


IOU  \
fold                                                                   0   
dataset_name split    model_type     num_epoch milestone_length            
coco         trainval iter_nwe_coatt 150       0.05              0.20004   
                                               0.1               0.19042   
                                               0.2               0.16092   
                                     250       0.05              0.22344   
                                               0.1               0.20236   
                                               0.2               0.20140   
                                     50        0.05              0.15838   
                                               0.1               0.14334   
                                               0.2               0.10102   

                                                                          \
fold                                                                   1   
dataset_name split    model_type     num_epoch milestone_length            
coco         trainval iter_nwe_coatt 150       0.05              0.20026   
                                               0.1               0.19240   
                                               0.2               0.16464   
                                     250       0.05              0.19560   
                                               0.1               0.19254   
                                               0.2               0.18700   
                                     50        0.05              0.16050   
                                               0.1               0.16198   
                                               0.2               0.15286   

                                                                          \
fold                                                                   2   
dataset_name split    model_type     num_epoch milestone_length            
coco         trainval iter_nwe_coatt 150       0.05              0.09080   
                                               0.1               0.09236   
                                               0.2               0.09832   
                                     250       0.05              0.11826   
                                               0.1               0.09526   
                                               0.2               0.10304   
                                     50        0.05              0.09598   
                                               0.1               0.08814   
                                               0.2               0.09482   

                                                                          \
fold                                                                   3   
dataset_name split    model_type     num_epoch milestone_length            
coco         trainval iter_nwe_coatt 150       0.05              0.09984   
                                               0.1               0.10292   
                                               0.2               0.10456   
                                     250       0.05              0.10484   
                                               0.1               0.10174   
                                               0.2               0.11226   
                                     50        0.05              0.11244   
                                               0.1               0.11488   
                                               0.2               0.11228   

                                                                     mIOU  
fold                                                                       
dataset_name split    model_type     num_epoch milestone_length            
coco         trainval iter_nwe_coatt 150       0.05              0.147735  
                                               0.1               0.144525  
                                     

In [6]:
exp_dir = "experiments/200104_204120_n_shots_fold_embed_type_bs_seed_num_epoch_lr_split_model_type_dataset_name_coco_iternwe_epochs_lr/*"

hp_summary = HyperParametersSummary(exp_dir, test_fname='final_test_miou_1.txt')

results_aggregated = hp_summary.table.groupby(['dataset_name', 'split', 'model_type', 'num_epoch', 'lr', 'fold']).mean().unstack()
results_aggregated['mIOU'] = results_aggregated.mean(axis=1)

results_aggregated

100%|██████████| 180/180 [00:00<00:00, 265.02it/s]

Parameters: num_epoch, n_shots, model_type, fold, seed, lr, split, bs, embed_type, dataset_name, IOU


IOU           \
fold                                                         0        1   
dataset_name split    model_type     num_epoch lr                         
coco         trainval iter_nwe_coatt 150       0.0005  0.09280  0.16492   
                                               0.001   0.13978  0.18634   
                                               0.002   0.14502  0.15620   
                                     250       0.0005  0.08748  0.16444   
                                               0.001   0.19274  0.18708   
                                               0.002   0.00000  0.10394   
                                     50        0.0005  0.11596  0.16498   
                                               0.001   0.10802  0.15252   
                                               0.002   0.12304  0.13856   

                                                                         \
fold                                                         2        3   
dataset_name split    model_type     num_epoch lr                         
coco         trainval iter_nwe_coatt 150       0.0005  0.09356  0.10992   
                                               0.001   0.09446  0.10468   
                                               0.002   0.12906  0.06066   
                                     250       0.0005  0.06224  0.10798   
                                               0.001   0.10402  0.11530   
                                               0.002   0.10574  0.05024   
                                     50        0.0005  0.10264  0.12704   
                                               0.001   0.09192  0.11228   
                                               0.002   0.12094  0.08072   

                                                           mIOU  
fold                                                             
dataset_name split    model_type     num_epoch lr                
coco         trainval iter_nwe_coatt 150       0.0005  0.115300  
                                               0.001   0.131315  
                                               0.002   0.122735  
                                     250       0.0005  0.105535  
                                               0.001   0.149785  
                                               0.002   0.064980  
                                     50        0.0005  0.127655  
                                               0.001   0.116185  
                                               0.002   0.115815

## PASCAL

In [8]:
exp_dir = "experiments/200113_145517_n_shots_fold_embed_type_bs_seed_milestone_length_split_model_type_dataset_name_pascal_1n5shots/*"

hp_summary = HyperParametersSummary(exp_dir, test_fname='final_test_miou_1.txt')

results_aggregated1 = hp_summary.table.groupby(['dataset_name', 'n_shots', 'model_type', 'fold']).mean().unstack()
results_aggregated1['mIOU'] = results_aggregated1.mean(axis=1)

100%|██████████| 160/160 [00:00<00:00, 3993.74it/s]

Parameters: milestone_length, n_shots, model_type, fold, seed, split, bs, embed_type, dataset_name, IOU


In [9]:
exp_dir = "experiments/200113_145517_n_shots_fold_embed_type_bs_seed_milestone_length_split_model_type_dataset_name_pascal_1n5shots/*"

hp_summary = HyperParametersSummary(exp_dir, test_fname='final_test_miou_5.txt')

results_aggregated5 = hp_summary.table.groupby(['dataset_name', 'n_shots', 'model_type', 'fold']).mean().unstack()
results_aggregated5['mIOU'] = results_aggregated5.mean(axis=1)
results_aggregated1.fillna(0.0)+results_aggregated5.fillna(0.0)

100%|██████████| 160/160 [00:00<00:00, 4385.43it/s]

Parameters: milestone_length, n_shots, model_type, fold, seed, split, bs, embed_type, dataset_name, IOU


IOU                             \
fold                                       0        1        2        3   
dataset_name n_shots model_type                                           
pascal       1       coatt           0.40418  0.57178  0.43566  0.36582   
                     iter_nwe_coatt  0.42510  0.64758  0.48116  0.46464   
                     nwe             0.44928  0.65242  0.47472  0.47286   
                     nwe_coatt       0.42622  0.65386  0.47590  0.45756   
             5       coatt           0.43398  0.63422  0.46002  0.43722   
                     iter_nwe_coatt  0.45890  0.65722  0.48610  0.46556   
                     nwe             0.45538  0.65436  0.47268  0.47510   
                     nwe_coatt       0.45698  0.65632  0.47850  0.44060   

                                         mIOU  
fold                                           
dataset_name n_shots model_type                
pascal       1       coatt           0.444360  
                     iter_nwe_coatt  0.504620  
                     nwe             0.512320  
                     nwe_coatt       0.503385  
             5       coatt           0.491360  
                     iter_nwe_coatt  0.516945  
                     nwe             0.514380  
                     nwe_coatt       0.508100

In [10]:
exp_dir = "experiments/200110_161548_n_shots_fold_embed_type_bs_seed_milestone_length_num_epoch_split_model_type_dataset_name_pascal_coatt_5shots/*"

hp_summary = HyperParametersSummary(exp_dir, test_fname='final_test_miou_5.txt')

results_aggregated = hp_summary.table.groupby(['dataset_name', 'model_type', 'embed_type', 'num_epoch', 'milestone_length', 'fold']).mean().unstack()
results_aggregated['mIOU'] = results_aggregated.mean(axis=1)
results_aggregated

100%|██████████| 160/160 [00:00<00:00, 437.41it/s]


Parameters: milestone_length, num_epoch, n_shots, model_type, fold, seed, split, bs, embed_type, dataset_name, IOU


IOU  \
fold                                                                     0   
dataset_name model_type     embed_type num_epoch milestone_length            
pascal       coatt          word2vec   100       0.1               0.42932   
                                                 0.2               0.42368   
                                       50        0.1               0.42694   
                                                 0.2               0.40090   
             iter_nwe_coatt word2vec   100       0.1               0.48416   
                                                 0.2               0.47898   
                                       50        0.1               0.47884   
                                                 0.2               0.44140   

                                                                            \
fold                                                                     1   
dataset_name model_type     embed_type num_epoch milestone_length            
pascal       coatt          word2vec   100       0.1               0.64154   
                                                 0.2               0.63666   
                                       50        0.1               0.63392   
                                                 0.2               0.62604   
             iter_nwe_coatt word2vec   100       0.1               0.65818   
                                                 0.2               0.65078   
                                       50        0.1               0.65774   
                                                 0.2               0.65210   

                                                                            \
fold                                                                     2   
dataset_name model_type     embed_type num_epoch milestone_length            
pascal       coatt          word2vec   100       0.1               0.45926   
                                                 0.2               0.45570   
                                       50        0.1               0.46420   
                                                 0.2               0.45862   
             iter_nwe_coatt word2vec   100       0.1               0.49018   
                                                 0.2               0.48734   
                                       50        0.1               0.48698   
                                                 0.2               0.48232   

                                                                            \
fold                                                                     3   
dataset_name model_type     embed_type num_epoch milestone_length            
pascal       coatt          word2vec   100       0.1               0.45340   
                                                 0.2               0.44810   
                                       50        0.1               0.43354   
                                                 0.2               0.41700   
             iter_nwe_coatt word2vec   100       0.1               0.46018   
                                                 0.2               0.45312   
                                       50        0.1               0.46252   
                                                 0.2               0.46856   

                                                                       mIOU  
fold                                                                         
dataset_name model_type     embed_type num_epoch milestone_length            
pascal       coatt          word2vec   100       0.1               0.495880  
                                                 0.2               0.491035  
                                       50        0.1               0.489650  
                                                 0.2               0.475640  
             iter_nwe_coatt word2vec   100       0.1               0.523175  
           